# Ground truth unit biological features

* author: steeve.laquitaine@epfl.ch

In [1]:
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import neurom as nm  # ground truth circuit details
import bluepy as bp  # ground truth circuit details
import spikeinterface as si
from neurom.view import plotly_impl

# move to PROJECT PATH
PROJ_PATH = "/gpfs/bbp.cscs.ch/project/proj85/home/laquitai/spikebias/"
os.chdir(PROJ_PATH)
from src.nodes.utils import get_config
from src.nodes.load import load_campaign_params

cfg, param_conf = get_config("silico_neuropixels", "concatenated").values()
SORTING_TRUE_PATH = cfg["sorting"]["simulation"]["ground_truth"]["output"]
BLUECFG = cfg["dataeng"]["blueconfig"]

# PARAMETERS

# SOMA
SOMA = {
    "soma": nm.core.types.NeuriteType.soma,
}
SOMA_FEATURES = [
    "radius",
    "volume",
]

# OTHER NEURITES
NEURITES_FEATURES = [
    "total_length",
    "total_height",
    "total_area",
    "number_of_sections",
    "section_lengths",
]
NEURITES = {
    "apical": nm.core.types.NeuriteType.apical_dendrite,
    "basal": nm.core.types.NeuriteType.basal_dendrite,
    "axon": nm.core.types.NeuriteType.axon,
}

2024-09-25 18:39:06,797 - root - utils.py - get_config - INFO - Reading experiment config.
2024-09-25 18:39:06,856 - root - utils.py - get_config - INFO - Reading experiment config. - done


### Custom functions

In [95]:
def get_features(morph_paths, feature_set, neurite_set):
    """Get morphometric features for all morphologies passed"""
    features = {"mtype": [], "feature_name": [], "feature_val": [], "loc": []}
    for mtype, paths in tqdm(morph_paths.items(), desc="mtypes"):
        for path in tqdm(
            paths, desc="morphologies", miniters=len(paths) / 100, leave=False
        ):
            morph = nm.load_morphology(path)
            for neurite_name, neurite_id in neurite_set.items():
                for feature in feature_set:
                    features["mtype"].append(mtype)
                    feature_name = (
                        "avg_section_length"
                        if feature == "section_lengths"
                        else feature
                    )
                    features["feature_name"].append(feature_name)
                    feature_val = nm.get(feature, morph, neurite_type=neurite_id)
                    feature_val = (
                        np.mean(feature_val)
                        if feature == "section_lengths"
                        else feature_val
                    )
                    features["feature_val"].append(feature_val)
                    features["loc"].append(neurite_name)
    df = pd.DataFrame.from_dict(features)
    df.drop(
        df.loc[(df["mtype"] == "L4_SSC") & (df["loc"] == "apical")].index, inplace=True
    )  # clear L4_SSC apicals...
    return df


def get_morph_paths(circuit, unit_ids: list):

    # get the units' mtypes and morphologies
    df = circuit.cells.get(unit_ids, properties=["mtype", "morphology"])

    # this creates the morphology path
    # for each unit by prefixing with the
    # common path of all morphologies
    # circuit.config["morphologies"]
    # to the morphology of the
    # units retrieved above
    # e.g., ../fixed_ais_L23PC_20201210/ascii/dend-C220797A-P3_axon-sm110131a1-3_IN.. for unit 12165
    # e.g., ../fixed_ais_L23PC_20201210/ascii/dend-rat_20160908_E3_LH2_cell2_axon-mt for unit 16652
    df["morph_path"] = (
        df["morphology"]
        .apply(lambda row: os.path.join(circuit.config["morphologies"], "%s.asc" % row))
        .astype(str)
    )
    df = df.drop(columns=["mtype", "morphology"])
    return df


def get_radius(circuit, unit_ids):

    # get the file paths of the morphology
    morph_paths = get_morph_paths(circuit, unit_ids)
    morph_paths = morph_paths.values.flatten()

    # get unit soma radius
    soma_radius = []
    for m_i in morph_paths:
        soma_radius.append(nm.load_morphology(m_i).soma.radius)
    return soma_radius

### Soma radius

In [96]:
# load once
circuit = bp.Simulation(BLUECFG).circuit

# get ground truth unit ids
SortingTrue = si.load_extractor(SORTING_TRUE_PATH)

In [97]:
# get radius
radius = get_radius(circuit, SortingTrue.unit_ids)

# unit-test
assert len(radius) == len(SortingTrue.unit_ids)
print(radius)

1388

### Neurites features

In [99]:
unit_id = SortingTrue.unit_ids[2]
print("unit:", unit_id)
morph_paths = get_morph_paths(circuit, [unit_id])
df = get_features(morph_paths, feature_set=NEURITES_FEATURES, neurite_set=NEURITES)
df

unit: 18371


mtypes: 1it [00:00,  6.56it/s]


,mtype,feature_name,feature_val,loc
0,morph_path,total_length,2677.573020,apical
1,morph_path,total_height,394.709137,apical
2,morph_path,total_area,5807.601221,apical
3,morph_path,number_of_sections,45.000000,apical
4,morph_path,avg_section_length,59.501621,apical
5,morph_path,total_length,2154.512926,basal
6,morph_path,total_height,265.234863,basal
7,morph_path,total_area,3809.369709,basal
8,morph_path,number_of_sections,32.000000,basal
9,morph_path,avg_section_length,67.328537,basal


# References

https://neurom.readthedocs.io/en/latest/features.html#